# Fine-Tuning Setup

This notebook contains the steps to fine-tune a pre-trained model on the soccer tactics dataset.

In [ ]:
# Import Required Libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Install required packages
!pip install gdown unsloth

In [ ]:
# Download dataset from Google Drive
import os

# Create a directory to store the dataset
os.makedirs('data', exist_ok=True)

# Use gdown to download files from the shared Google Drive folder
!gdown --folder https://drive.google.com/drive/folders/1izgyjcd4k6V7txzYcgtx5-cDzTdfRAmN?usp=drive_link -O data

print("Dataset downloaded and saved to 'data' folder.")

In [ ]:
# Load Dataset from downloaded files
dataset = load_dataset('json', data_files={'train': 'data/football_train.jsonl', 'validation': 'data/football_val.jsonl'})
print(dataset)

In [ ]:
# Define Model and Tokenizer
model_name = 'unsunsloth/Qwen3-4B-Instruct-2507-GGUFloth'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

In [ ]:
# Preprocess Data
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [ ]:
# Fine-Tune the Model using unsloth
from unsloth import Trainer as UnslothTrainer, TrainingArguments as UnslothTrainingArguments

training_args = UnslothTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir='./logs',
)

trainer = UnslothTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
)

trainer.train()

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Save the Fine-Tuned Model to Google Drive
model.save_pretrained('/content/drive/My Drive/fine_tuned_model')
tokenizer.save_pretrained('/content/drive/My Drive/fine_tuned_model')